#Getting Started
This notebook provides a basic introduction to building a LangGraph, a framework for creating complex conversational AI agents. It covers a basic conversational loop where the assistant generates a response, a condition checks if a tool is needed, and if so, a tool is invoked, and its output is fed back to the assistant. The visualization helps in understanding this flow and the decision points within the graph.

In [ ]:
!pip install -q langchain-google-genai google-generativeai langgraph langchain_community ddgs

In [ ]:
import google.generativeai as genai
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent
from typing import Annotated
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.prebuilt import InjectedState
from langgraph.graph import StateGraph, START, MessagesState
from langgraph.types import Command

# Get the API key from Colab secrets
API_KEY = userdata.get('GOOGLE_API_KEY')

# Configure the genai library with the API key
genai.configure(api_key=API_KEY)

### Authenticate your notebook environment (Colab only)

In [ ]:
from google.colab import auth
auth.authenticate_user()

### Test your API Key

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=API_KEY)
try:
    response = llm.invoke("What is LLM?")
    print(response)
except Exception as e:
    print(f"An error occurred: {e}")


### Create a Search Tool

In [ ]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
def search_tool(prompt: str)->str:
  "Performs Search using DuckDuckGo"
  search = DuckDuckGoSearchRun()
  text=search.invoke(prompt)
  return text


In [ ]:
search_tool("What is capital city of India ?")

### Create a Reserach Agent

In [ ]:
from langgraph.prebuilt import create_react_agent
# The 'create_react_agent' function in LangGraph's pre-built templates provides
# a streamlined way to create a ReAct agent. ReAct (Reasoning and Acting) is a popular framework
# for building agents that can interact with tools.
# This function encapsulates the core components and logic needed to set up such an agent.
research_agent = create_react_agent(llm, tools=[search_tool], prompt="You are a researcher. DO NOT do any math.",name="research_agent")

In [ ]:
from langchain_core.messages import convert_to_messages


def pretty_print_message(message, indent=False):
    pretty_message = message.pretty_repr(html=True)
    if not indent:
        print(pretty_message)
        return

    indented = "\n".join("\t" + c for c in pretty_message.split("\n"))
    print(indented)


def pretty_print_messages(update, last_message=False):
    is_subgraph = False
    if isinstance(update, tuple):
        ns, update = update
        # skip parent graph updates in the printouts
        if len(ns) == 0:
            return

        graph_id = ns[-1].split(":")[0]
        print(f"Update from subgraph {graph_id}:")
        print("\n")
        is_subgraph = True

    for node_name, node_update in update.items():
        update_label = f"Update from node {node_name}:"
        if is_subgraph:
            update_label = "\t" + update_label

        print(update_label)
        print("\n")

        messages = convert_to_messages(node_update["messages"])
        if last_message:
            messages = messages[-1:]

        for m in messages:
            pretty_print_message(m, indent=is_subgraph)
        print("\n")

### Test Research Agent

In [ ]:
for chunk in research_agent.stream(
    {"messages": [{"role": "user", "content": "What is Capital City of India ?"}]}
):
    pretty_print_messages(chunk)

### Create Python Functions as tools

In [ ]:
def add(a: float, b: float):
    """Add two numbers."""
    return a + b


def multiply(a: float, b: float):
    """Multiply two numbers."""
    return a * b


def divide(a: float, b: float):
    """Divide two numbers."""
    return a / b


### Create Math Agent

In [ ]:
math_agent = create_react_agent(
    model=llm,
    tools=[add, multiply, divide],
    prompt=(
        "You are a math agent.\n\n"
        "INSTRUCTIONS:\n"
        "- Assist ONLY with math-related tasks\n"
        "- After you're done with your tasks, respond to the supervisor directly\n"
        "- Respond ONLY with the results of your work, do NOT include ANY other text."
    ),
    name="math_agent",
)

### Test Match Agent

In [ ]:
for chunk in math_agent.stream(
    {"messages": [{"role": "user", "content": "what's (3 + 5) x 7"}]}
):
    pretty_print_messages(chunk)

### Create Agents as Tools for Supervisor Agent

In [ ]:
def create_handoff_tool(*, agent_name: str, description: str | None = None):
    name = f"transfer_to_{agent_name}"
    description = description or f"Ask {agent_name} for help."

    @tool(name, description=description)
    def handoff_tool(
        state: Annotated[MessagesState, InjectedState],
        tool_call_id: Annotated[str, InjectedToolCallId],
    ) -> Command:
        tool_message = {
            "role": "tool",
            "content": f"Successfully transferred to {agent_name}",
            "name": name,
            "tool_call_id": tool_call_id,
        }
        return Command(
            goto=agent_name,
            update={**state, "messages": state["messages"] + [tool_message]},
            graph=Command.PARENT,
        )

    return handoff_tool

In [ ]:
# Handoffs
assign_to_research_agent = create_handoff_tool(
    agent_name="research_agent",
    description="Assign task to a researcher agent.",
)

assign_to_math_agent = create_handoff_tool(
    agent_name="math_agent",
    description="Assign task to a math agent.",
)

### Building Supervisor Agent

In [ ]:
supervisor_agent = create_react_agent(
    model=llm,
    tools=[assign_to_research_agent, assign_to_math_agent],
    prompt=(
        "You are a supervisor managing two agents:\n"
        "- a research agent. Assign research-related tasks to this agent\n"
        "- a math agent. Assign math-related tasks to this agent\n"
        "Assign work to one agent at a time, do not call agents in parallel.\n"
        "Do not do any work yourself."
    ),
    name="supervisor",
)

### Defining the Graph

In [ ]:
from langgraph.graph import END

# Define the multi-agent supervisor graph
supervisor = (
    StateGraph(MessagesState)
    # NOTE: `destinations` is only needed for visualization and doesn't affect runtime behavior
    .add_node(supervisor_agent, destinations=("research_agent", "math_agent", END))
    .add_node(research_agent)
    .add_node(math_agent)
    .add_edge(START, "supervisor")
    # always return back to the supervisor
    .add_edge("research_agent", "supervisor")
    .add_edge("math_agent", "supervisor")
    .compile()
)

In [ ]:
from IPython.display import display, Image

display(Image(supervisor.get_graph().draw_mermaid_png()))

### Test the Agent

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "find US and New York state GDP in 2024. what % of US GDP was New York state?",
            }
        ]
    },
):
    pretty_print_messages(chunk, last_message=True)

final_message_history = chunk["supervisor"]["messages"]